In [7]:
import numpy as np

# you must use python 3.6, 3.7, 3.8, 3.9 for sourcedefender
import sourcedefender
from HomeworkFramework import Function

class RS_optimizer(Function): # need to inherit this class "Function"
    def __init__(self, target_func):
        super().__init__(target_func) # must have this init to work normally

        self.lower = self.f.lower(target_func)
        self.upper = self.f.upper(target_func)
        self.dim = self.f.dimension(target_func)

        self.target_func = target_func

        self.eval_times = 0
        self.optimal_value = float("inf")
        self.optimal_solution = np.empty(self.dim)
        print(self.lower, self.upper, self.dim)
        print("bang------------------bang")
        

    def get_optimal(self):
        return self.optimal_solution, self.optimal_value

    def run(self, FES): # main part for your implementation
        
        while self.eval_times < FES:
            #print('=====================FE=====================')
            #print(self.eval_times)

            solution = np.random.uniform(np.full(self.dim, self.lower), np.full(self.dim, self.upper), self.dim)
            value = self.f.evaluate(func_num, solution)
            self.eval_times += 1

            if value == "ReachFunctionLimit":
                #print("ReachFunctionLimit")
                break            
            if float(value) < self.optimal_value:
                self.optimal_solution[:] = solution
                self.optimal_value = float(value)

            #print("optimal: {}\n".format(self.get_optimal()[1]))
            

if __name__ == '__main__':
    func_num = 1
    fes = 0
    #function1: 1000, function2: 1500, function3: 2000, function4: 2500
    while func_num < 5:
        if func_num == 1:
            fes = 1000
        elif func_num == 2:
            fes = 1500
        elif func_num == 3:
            fes = 2000 
        else:
            fes = 2500

        # you should implement your optimizer
        op = RS_optimizer(func_num)
        op.run(fes)
        
        best_input, best_value = op.get_optimal()
        print(best_input, best_value)
        
        '''
        # change the name of this file to your student_ID and it will output properlly
        with open("{}_function{}.txt".format(__file__.split('_')[0], func_num), 'w+') as f:
            for i in range(op.dim):
                f.write("{}\n".format(best_input[i]))
            f.write("{}\n".format(best_value))
        '''
        func_num += 1 


-1 1 6
bang------------------bang
[ 0.33467767 -0.69844363  0.66841314 -0.9437147   0.60852932 -0.03589813] 0.057789478176996534
-10 10 2
bang------------------bang
[1.34179027 1.00463998] 0.12340481544047759
-32.768 32.768 5
bang------------------bang
[ 8.01993447 -2.95956915 -0.09598868  5.27702994  6.8735043 ] 12.520738679968943
-600 600 10
bang------------------bang
[ -12.87421087  235.3897673   -54.235789    166.45863032  367.34968225
 -221.20026626  -89.67401832  -31.11757845  -86.63021111  201.20243413] 61.36633859236563


In [14]:
class a:
    # 駕駛方法
    def drive(self):
        print("a drive method is called.")

# 交通工具(基底類別)
class Transportation(a):
    # 駕駛方法
    def drive(self, color):
        self.color = color
        print("Base class drive method is called.", self.color)
# 汽車子類別
class Car(Transportation):
    # 駕駛方法
    def drive(self):
        super().drive("girl")
        print("Sub class drive method is called.")
mazda = Car()
mazda.drive()
b = np.full(5, 2)
print(b)

Base class drive method is called. girl
Sub class drive method is called.
[2 2 2 2 2]


In [9]:
import numpy as np
import sourcedefender
from HomeworkFramework import Function
import math

class CMAES_optimizer(Function):
    def __init__(self, target_func):
        super().__init__(target_func)
        # number of objective variables/problem dimension
        self.dim = self.f.dimension(target_func) 
        self.lower = self.f.lower(target_func)
        self.upper = self.f.upper(target_func)
         
        self.target_func = target_func
        self.eval_times = 0 # equivalent to g , g is number of generations
        self.optimal_value = float("inf")
        self.optimal_solution = np.empty(self.dim)
        self.setting()
        
    def setting(self):
        #strategy parameter----------------
        # objective variables initial point
        self.xmean = np.copy(np.random.random(self.dim))
        
        # coordinate wise standard deviation (step-size)
        self.sigma = 0.25
        # population size, offspring number
        self.Lambda = 5 + int(3 * np.log(self.dim))
        self.mu = int(self.Lambda / 2)
        
        self.weights = np.array([np.log(self.mu + 0.5) - np.log(i + 1) for i in range(self.mu)])
        # normalize recombination weights array
        self.weights = np.array([w / sum(self.weights) for w in self.weights])
        # variance-effective size of mu

        self.mueff = 1 / np.sum(np.power(w, 2) for w in self.weights)
        #strategy parameter----------------------------------
        # time constant for cumulation for C
        self.cc = (4 + self.mueff / self.dim) / (self.dim + 3 + 2 * self.mueff / self.dim)
        # t-const for cumulation for sigma control
        self.cs = (self.mueff + 6) / (self.dim + self.mueff + 4)
        # learning rate for rank-one update of C
        self.c1 = 2 / ((self.dim + 1.3) ** 2 + self.mueff)
        # for rank-mu update
        self.cmu = min([1 - self.c1, 2 * (self.mueff - 2 + 1 / self.mueff) / ((self.dim + 2) ** 2 + self.mueff)])
        # damping for sigma
        self.damps = 1 + self.cs + 2 * max([0, ((self.mueff - 1) / self.dim) ** 0.5 - 1])
        # init dynamic (internal) strategy parameters and constants------------------------------
        # evolution paths for C and sigma
        self.pc, self.ps = np.zeros(self.dim), np.zeros(self.dim)
        # B defines the coordinate system, diagonal matrix D defines the scaling covariance matrix.
        self.B, self.C, self.M = np.eye(self.dim), np.eye(self.dim), np.eye(self.dim)
        self.D = np.ones(self.dim)
    
    def fit(self):
        fitness = np.zeros((self.Lambda, self.dim))
        arz = np.zeros((self.Lambda, self.dim))
        arx = np.zeros((self.Lambda, self.dim))
        fitvals = np.zeros(self.Lambda)
        for i in range(self.Lambda):
            arz[i] = np.random.normal(0, 1, self.dim)
            arx[i] = np.dot(self.B * self.D, arz[i])
            fitness[i] = self.xmean + self.sigma * arx[i]
            fitness[i] = np.clip(fitness[i], self.lower, self.upper)
            
            value = self.f.evaluate(func_num, fitness[i])
            self.eval_times += 1
            
            if value == "ReachFunctionLimit":
                print("ReachFunctionLimit")
                break
                
            fitvals[i] = value
            if fitvals[i] < self.optimal_value:
                self.optimal_value = fitvals[i]
                self.optimal_solution = fitness[i]
        return fitness, arz, arx, fitvals, value
    
    def step(self):
        # Sample
        self.D, self.B = np.linalg.eigh(self.C)
        self.D = self.D** 0.5
        
        fitness, arz, arx, fitvals, value = self.fit()

        if value == "ReachFunctionLimit":
            return
        # sort and update mean
        argx = np.argsort(fitvals)

        self.xmean = self.xmean + self.sigma * np.sum(self.weights[i] * arx[argx[i]] for i in range(self.mu))

        # update evolution path
        ps_bias = np.sum(self.weights[i] * arz[argx[i]] for i in range(self.mu))*np.sqrt(self.cs * (2 - self.cs) * self.mueff)
        self.ps = self.ps - self.cs * self.ps + ps_bias
        
        pc_bias = np.sum(self.weights[i] * arx[argx[i]] for i in range(self.mu))*np.sqrt(self.cc * (2 - self.cc) * self.mueff)
        self.pc = self.pc - self.cc * self.pc + pc_bias

        # update covariance matrix C
        part1 = (1 - self.c1 - self.cmu) * self.C
        part2 = self.c1 * np.dot(self.pc.reshape(self.dim, 1), self.pc.reshape(1, self.dim))
        part3 = np.zeros((self.dim, self.dim))
        for i in range(self.mu):
            part3 += self.cmu * self.weights[i] * np.dot(arx[argx[i]].reshape(self.dim, 1), arx[argx[i]].reshape(1, self.dim))
        self.C = part1 + part2 + part3

        # update step-size
        self.sigma *= np.exp((self.cs / 2) * (np.sum(np.power(x, 2) for x in self.ps) / self.dim - 1))

    def run(self, FES):
        while self.eval_times < FES:
            #print('=====================FE=====================')
            #print(self.eval_times)
            self.step()
            #print("optimal: {}\n".format(self.get_optimal()[1]))

    def get_optimal(self):
        return self.optimal_solution, self.optimal_value

if __name__ == '__main__':
    func_num = 1
    fes = 0
    #function1: 1000, function2: 1500, function3: 2000, function4: 2500
    while func_num < 5:
        if func_num == 1:
            fes = 1000
        elif func_num == 2:
            fes = 1500
        elif func_num == 3:
            fes = 2000 
        else:
            fes = 2500

        # you should implement your optimizer
        op =  CMAES_optimizer(func_num)
        op.run(fes)
        
        best_input, best_value = op.get_optimal()
        print(best_input, best_value)
        
        # change the name of this file to your student_ID and it will output properlly
        with open("{}_function{}.txt".format(__file__.split('_')[0], func_num), 'w+') as f:
            for i in range(op.dim):
                f.write("{}\n".format(best_input[i]))
            f.write("{}\n".format(best_value))
        func_num += 1 

C:\Users\a0985\anaconda3\lib\site-packages\ipykernel_launcher.py:35: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
C:\Users\a0985\anaconda3\lib\site-packages\ipykernel_launcher.py:86: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
C:\Users\a0985\anaconda3\lib\site-packages\ipykernel_launcher.py:89: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
C:\Users\a0985\anaconda3\lib\site-packages\ipykernel_launcher.py:93: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
C:\Users

=====================FE=====================
0
optimal: 2.84485568976309

=====================FE=====================
10
optimal: 2.022947085304245

=====================FE=====================
20
optimal: 2.022947085304245

=====================FE=====================
30
optimal: 1.407477346074176

=====================FE=====================
40
optimal: 0.9115836179530553

=====================FE=====================
50
optimal: 0.2421394557207793

=====================FE=====================
60
optimal: 0.0870630856158762

=====================FE=====================
70
optimal: 0.07029370837005249

=====================FE=====================
80
optimal: 0.07029370837005249

=====================FE=====================
90
optimal: 0.005371555390523347

=====================FE=====================
100
optimal: 0.005371555390523347

=====================FE=====================
110
optimal: 0.005371555390523347

=====================FE=====================
120
optimal: 0.005371555390

NameError: name '__file__' is not defined